Copyright Preferred Computational Chemistry, Inc. and Preferred Networks, Inc. as contributors to Matlantis contrib project

This document has been machine translated from Japanese to English.

# Example of Generating Random Structures on Matlantis
*Please note that the larger the number of atoms, the longer the runtime will be.

In [1]:
!pip install ase pymatgen

# Please install the libraries only at the first usage.

## 1. Random Structure Generation using ASE

In [1]:
import numpy as np
from ase import Atoms
from ase.data import atomic_numbers
from ase.ga.utilities import closest_distances_generator, CellBounds
from ase.data import atomic_numbers, covalent_radii
from ase.ga.startgenerator import StartGenerator
from ase.ga.utilities import closest_distances_generator
from ase.visualize import view

### 1-1. Crystal

In [2]:
blocks = ['Ti'] * 4 + ['O'] * 8 # Composition of the crystal you want to make
box_volume = 12 * 12 # Empirically, 10~12 times the number of atoms tends to work well

blmin = closest_distances_generator(atom_numbers=Atoms(blocks).get_atomic_numbers(),
                                    ratio_of_covalent_radii=0.8) # List of distances at which atoms can get close to each other
cellbounds = CellBounds(bounds={'phi': [30, 150], 'chi': [30, 150],
                                'psi': [30, 150], 'a': [3, 10],
                                'b': [3, 10], 'c': [3, 10]}) # Range of motion for the cell

slab = Atoms('', pbc=True) # Prepare a template to pack atoms
sg = StartGenerator(slab, blocks, blmin, box_volume=box_volume,
                    cellbounds=cellbounds, 
                    number_of_variable_cell_vectors=3,
                    test_too_far=False)

In [3]:
atoms = sg.get_new_candidate()
atoms

Atoms(symbols='Ti4O8', pbc=True, cell=[[3.399468661931798, 0.0, 0.0], [-0.7876779702811255, 5.57401653937108, 0.0], [-2.4380262757553357, -1.7769862187552858, 7.59946811350224]], tags=...)

In [4]:
v = view(atoms, viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

### 1-2. Molecular Crystal

In [5]:
blocks = ['H2O'] * 8
box_volume = 20 * 8 # Depends on how densely you want to pack

blmin = closest_distances_generator(atom_numbers=[1,8], # H and O
                                    ratio_of_covalent_radii=1.2) # Taking a larger value makes it less likely for molecules to stick together
cellbounds = CellBounds(bounds={'phi': [30, 150], 'chi': [30, 150],
                                'psi': [30, 150], 'a': [3, 10],
                                'b': [3, 10], 'c': [3, 10]})

slab = Atoms('', pbc=True)
sg = StartGenerator(slab, blocks, blmin, box_volume=box_volume,
                    cellbounds=cellbounds, 
                    number_of_variable_cell_vectors=3,
                    test_too_far=False)

In [6]:
atoms = sg.get_new_candidate()
atoms

Atoms(symbols='OH2OH2OH2OH2OH2OH2OH2OH2', pbc=True, cell=[[5.351229037091787, 0.0, 0.0], [0.4781589609694113, 5.361917945042938, 0.0], [-0.2782740527196683, -1.1820004956289019, 5.576301915370867]], tags=...)

In [7]:
v = view(atoms, viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

## 2. Random Structure Generation using pyxtal

The pyxtal library (https://github.com/qzhu2017/PyXtal) is open-source software released under the MIT license. Development is still active, so please be cautious about which version you use.

pyxtal.XRD depends on `numba>=0.50.1`, and `numba 0.54.1` requires `numpy<1.21,>=1.17`, thus it cannot be installed with Matlantis's default numpy version.

!pip install numpy==1.20
!pip install pyxtal

In [9]:
from pyxtal import pyxtal
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

In [11]:
!pip install -U numpy

### 2-1. Crystal

Generates structures while maintaining symmetry by specifying the space group by its number.\
For example, the rutile type belongs to P42/mnm (136).

In [12]:
crystal = pyxtal()
crystal.from_random(3, 136, ['Ti','O'], [4,8]) # Dimension, space group, elements, composition
crystal


------Crystal from random------
Dimension: 3
Composition: O8Ti4
Group: P42/mnm (136)
tetragonal lattice:   5.9228   5.9228   4.1791  90.0000  90.0000  90.0000
Wyckoff sites:
	Ti @ [ 0.2499 -0.2499  0.0000], WP [4g] Site [m.m2]
	 O @ [ 0.0000  0.5000  0.2500], WP [4d] Site [-4..]
	 O @ [ 0.0000  0.0000  0.0000], WP [2a] Site [2/m.2/m2/m]
	 O @ [ 0.0000  0.0000  0.5000], WP [2b] Site [2/m.2/m2/m]

In [13]:
atoms = crystal.to_ase()
atoms.wrap()
v = view(atoms, viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

### 2-2. Molecular Crystal

In [14]:
mol_crystal = pyxtal(molecular=True)
mol_crystal.from_random(3, 36, ['H2O'], [8])
mol_crystal


------Crystal from random------
Dimension: 3
Composition: [H2O]8
Group: Cmc21 (36)
orthorhombic lattice:   7.2906   4.8405  11.3199  90.0000  90.0000  90.0000
Wyckoff sites:
	H2O1         @ [ 0.7033  0.3840  0.9518]  WP [8b] Site [1] Euler [-138.0    7.9   21.3]

In [15]:
v = view(mol_crystal.to_ase(), viewer='ngl')
v.view.add_representation("ball+stick")
display(v)

### 2-3. Cluster

Set the dimension to 0 and specify using a point group instead of a space group. If no symmetry is to be introduced, use C1 (1).

In [16]:
cluster = pyxtal()
cluster.from_random(0, 1, ['Pt'], [13]) # Dimension, space group, elements, composition
cluster


------Crystal from random------
Dimension: 0
Composition: Pt13
Group: C1 (1)
spherical lattice:   3.6669   3.6669   3.6669  90.0000  90.0000  90.0000
Wyckoff sites:
	Pt @ [-0.7798  0.3045 -0.2505], WP [1a] Site [1]
	Pt @ [-0.2504  0.0355 -0.0971], WP [1a] Site [1]
	Pt @ [ 0.1569 -0.0117  0.5000], WP [1a] Site [1]
	Pt @ [-0.2366 -0.3016 -0.2748], WP [1a] Site [1]
	Pt @ [ 0.6771 -0.6440  0.1037], WP [1a] Site [1]
	Pt @ [ 0.4976 -0.7905 -0.2079], WP [1a] Site [1]
	Pt @ [ 0.3236  0.7859 -0.3149], WP [1a] Site [1]
	Pt @ [ 0.7292  0.4299  0.4916], WP [1a] Site [1]
	Pt @ [-0.4418 -0.1537 -0.5743], WP [1a] Site [1]
	Pt @ [ 0.6301 -0.0632  0.5775], WP [1a] Site [1]
	Pt @ [ 0.5740 -0.1216 -0.7349], WP [1a] Site [1]
	Pt @ [ 0.6156  0.1639  0.2582], WP [1a] Site [1]
	Pt @ [-0.1879  0.2728  0.4535], WP [1a] Site [1]

In [17]:
v = view(cluster.to_ase(), viewer='ngl')
v.view.add_representation("ball+stick")
display(v)